<a href="https://colab.research.google.com/github/luanps/pygaggle/blob/master/Run_pygaggle_monoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# monoBERT Passage Ranking on MSMARCO


## Install dependencies

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!git clone --recursive https://github.com/luanps/pygaggle.git 
!mv pygaggle pygaggle_repo && mv pygaggle_repo/* ./
!rmdir pygaggle_repo
!pip install -e .

In [ ]:
#Download MSMARCO Passage data - eval

!wget https://www.dropbox.com/s/5xa5vjbjle0c8jv/msmarco_ans_small.zip -P data
!unzip data/msmarco_ans_small.zip -d data

--2022-02-23 20:58:11--  https://www.dropbox.com/s/5xa5vjbjle0c8jv/msmarco_ans_small.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/5xa5vjbjle0c8jv/msmarco_ans_small.zip [following]
--2022-02-23 20:58:11--  https://www.dropbox.com/s/raw/5xa5vjbjle0c8jv/msmarco_ans_small.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8faa2e804f84693fa393322350.dl.dropboxusercontent.com/cd/0/inline/BgRbhf-1HcgiRKZS3oweoU5ck0osegTJVD3Ez5srEu4yxcWJtjNco1UAWOyJFTkPD6nVylqlCDn7QDKJYCUE9ntOtErQ6lBtlNSJV9Pk5VsOqQ0E63ngkpsVrJtSwcIg3d3jA3zc1KiVMC8RAfnvCBvI/file# [following]
--2022-02-23 20:58:12--  https://uc8faa2e804f84693fa393322350.dl.dropboxusercontent.com/cd/0/inline/BgRbhf-1HcgiRKZS3oweoU5ck0osegTJVD3Ez5srEu4yxcWJtjNco1UAWOyJF

In [ ]:
#Download pre-built indexes

!wget https://git.uwaterloo.ca/jimmylin/anserini-indexes/raw/master/index-msmarco-passage-20191117-0ed488.tar.gz -P indexes
!tar xvfz indexes/index-msmarco-passage-20191117-0ed488.tar.gz -C indexes

--2022-02-23 20:58:13--  https://git.uwaterloo.ca/jimmylin/anserini-indexes/raw/master/index-msmarco-passage-20191117-0ed488.tar.gz
Resolving git.uwaterloo.ca (git.uwaterloo.ca)... 129.97.83.4
Connecting to git.uwaterloo.ca (git.uwaterloo.ca)|129.97.83.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2153209812 (2.0G) [application/x-gzip]
Saving to: ‘indexes/index-msmarco-passage-20191117-0ed488.tar.gz’

index-msmarco-passa 100%[===================>]   2.00G  36.9MB/s    in 68s     

2022-02-23 20:59:21 (30.4 MB/s) - ‘indexes/index-msmarco-passage-20191117-0ed488.tar.gz’ saved [2153209812/2153209812]

index-msmarco-passage-20191117-0ed488/
index-msmarco-passage-20191117-0ed488/_6.fnm
index-msmarco-passage-20191117-0ed488/_6.fdx
index-msmarco-passage-20191117-0ed488/_4.si
index-msmarco-passage-20191117-0ed488/_7.fdx
index-msmarco-passage-20191117-0ed488/_7.fnm
index-msmarco-passage-20191117-0ed488/_5.fnm
index-msmarco-passage-20191117-0ed488/_5.fdx
index-msmar

## monoBERT inference

In [ ]:
#Eval baseline BM25 Retrieval

!python tools/scripts/msmarco/msmarco_passage_eval.py data/msmarco_ans_small/qrels.dev.small.tsv data/msmarco_ans_small/run.dev.small.tsv

### Dense retrieval with monoBERT


In [ ]:
!python -um pygaggle.run.evaluate_passage_ranker --split dev \
                                                --method seq_class_transformer \
                                                --model castorini/monobert-large-msmarco \
                                                --dataset data/msmarco_ans_small/ \
                                                --index-dir indexes/index-msmarco-passage-20191117-0ed488 \
                                                --task msmarco \
                                                --output-file run.monobert.ans_small.dev.tsv

In [ ]:
!gsutil cp run.monobert.ans_small.dev.tsv gs://luanps/information_retrieval/pygaggle/monobert/

#### Evaluation

In [ ]:
#MRR Eval
!python pygaggle/tools/scripts/msmarco/msmarco_passage_eval.py data/msmarco_ans_small/qrels.dev.small.tsv run.monobert.ans_small.dev.tsv >> monobert_mrr_eval.txt


In [ ]:
!gsutil cp monobert_mrr_eval.txt  gs://luanps/information_retrieval/pygaggle/monobert/

In [ ]:
#TREC Eval
!python -m pyserini.eval.convert_msmarco_run_to_trec_run --input run.msmarco-passage.tct_colbert.bf.tsv \
                                                         --output run.msmarco-passage.tct_colbert.bf.trec

!python -m pyserini.eval.trec_eval -c -mrecall.1000 \
                                      -mmap msmarco-passage-dev-subset \
                                      run.msmarco-passage.tct_colbert.bf.trec >> bf_trec_eval.txt

## Extract monoBERT vector representations on MS-MARCO training set

### Download MSMARCO Passage data

In [ ]:
!wget https://msmarco.blob.core.windows.net/msmarcoranking/qrels.train.tsv 
!wget https://msmarco.blob.core.windows.net/msmarcoranking/queries.tar.gz 

!gsutil cp  gs://luanps/monoBERT/msmarco/triples.train.small.tar.gz  .
!mkdir data/msmarco_train/

!tar -xzvf triples.train.small.tar.gz
!mv triples.*tsv data/msmarco_train/
!rm triples.train.small.tar.gz

!tar -xzvf queries.tar.gz
!mv queries*tsv qrels.train.tsv data/msmarco_train/
!rm queries.tar.gz

--2022-02-23 21:00:02--  https://msmarco.blob.core.windows.net/msmarcoranking/qrels.train.tsv
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10589532 (10M) [text/tab-separated-values]
Saving to: ‘qrels.train.tsv’

qrels.train.tsv     100%[===================>]  10.10M  6.89MB/s    in 1.5s    

2022-02-23 21:00:04 (6.89 MB/s) - ‘qrels.train.tsv’ saved [10589532/10589532]

--2022-02-23 21:00:04--  https://msmarco.blob.core.windows.net/msmarcoranking/queries.tar.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18882551 (18M) [application/gzip]
Saving to: ‘queries.tar.gz’

queries.tar.gz     

### Run model inference on Train set

In [ ]:
from typing import Optional, List, Tuple
from pathlib import Path
import logging
import os

from pydantic import BaseModel, validator
from transformers import (AutoModel,
                          AutoTokenizer,
                          BertForSequenceClassification)
import torch

#from .args import ArgumentParserBuilder, opt
from pygaggle.rerank.base import Reranker
from pygaggle.rerank.bm25 import Bm25Reranker
from pygaggle.rerank.transformer import (
    UnsupervisedTransformerReranker,
    MonoT5,
    DuoT5,
    MonoBERT
)
from pygaggle.rerank.random import RandomReranker
from pygaggle.rerank.similarity import CosineSimilarityMatrixProvider
from pygaggle.model import (SimpleBatchTokenizer,
                            RerankerEvaluator,
                            DuoRerankerEvaluator,
                            metric_names,
                            MsMarcoWriter)
from pygaggle.data import MsMarcoDataset
from pygaggle.settings import MsMarcoSettings


SETTINGS = MsMarcoSettings()
METHOD_CHOICES = ('transformer', 'bm25', 't5', 'seq_class_transformer',
                  'random', 'duo_t5')

In [ ]:
from pygaggle.run.evaluate_passage_ranker import PassageRankingEvaluationOptions


def construct_transformer(options:
                          PassageRankingEvaluationOptions) -> Reranker:
    device = torch.device(options.device)
    model = AutoModel.from_pretrained(options.model,
                                      from_tf=options.from_tf).to(device).eval()
    tokenizer = SimpleBatchTokenizer(AutoTokenizer.from_pretrained(
        options.tokenizer_name, use_fast=False),
        options.batch_size)
    provider = CosineSimilarityMatrixProvider()
    return UnsupervisedTransformerReranker(model, tokenizer, provider)


def construct_seq_class_transformer(options: PassageRankingEvaluationOptions
                                    ) -> Reranker:
    try:
        model = MonoBERT.get_model(
            options.model, from_tf=options.from_tf, device=options.device, output_hidden_states=True)
    except AttributeError:
        # Hotfix for BioBERT MS MARCO. Refactor.
        BertForSequenceClassification.bias = torch.nn.Parameter(
            torch.zeros(2))
        BertForSequenceClassification.weight = torch.nn.Parameter(
            torch.zeros(2, 768))
        model = BertForSequenceClassification.from_pretrained(
            options.model, from_tf=options.from_tf)
        model.classifier.weight = BertForSequenceClassification.weight
        model.classifier.bias = BertForSequenceClassification.bias
        device = torch.device(options.device)
        model = model.to(device).eval()
    tokenizer = MonoBERT.get_tokenizer(options.tokenizer_name)
    return MonoBERT(model, tokenizer)


def construct_bm25(options: PassageRankingEvaluationOptions) -> Reranker:
    return Bm25Reranker(index_path=str(options.index_dir))


In [ ]:
args = {
    'task': 'msmarco',
    'dataset': 'data/msmarco_train',
    'index_dir': 'indexes/index-msmarco-passage-20191117-0ed488',
    'method': 'seq_class_transformer',
    'model': 'castorini/monobert-large-msmarco', 
    'duo_model': '',
    'mono_hits': 50,
    'output_file': 'tmp',
    'mono_cache_write_path': '.', 
    'mono_cache_load_path': '.', 
    'overwrite_output': False,
    'split': 'train',
    'batch_size': 96,
    'device': 'cuda:0',
    'is_duo': False,
    'from_tf': False,
    'metrics': ['precision@1', 'recall@3', 'recall@50', 'recall@1000', 'mrr', 'mrr@10'],
    'model_type': None,
    'tokenizer_name': None
 }

In [ ]:
#!wget https://msmarco.blob.core.windows.net/msmarcoranking/triples.train.small.tar.gz

In [ ]:
#!mv data/msmarco_train/qrels.train.tsv data/msmarco_train/qrels.train.small.tsv
#!mv data/msmarco_train/queries.train.tsv data/msmarco_train/queries.train.small.tsv

In [ ]:
import os
from collections import OrderedDict, defaultdict
from pathlib import Path
from typing import List, Set, DefaultDict
import logging
from itertools import permutations

from pydantic import BaseModel
import scipy.special as sp
import numpy as np
from tqdm import tqdm

from pygaggle.data.relevance import RelevanceExample, MsMarcoPassageLoader
from pygaggle.rerank.base import Query, Text
from pygaggle.data.unicode import convert_to_unicode


__all__ = ['MsMarcoExample', 'MsMarcoDataset']


# MsMarcoExample represents a query along with its ranked and re-ranked candidates.
class MsMarcoExample(BaseModel):
    qid: str
    text: str
    candidates: List[str]
    relevant_candidates: Set[str]


class MsMarcoDataset(BaseModel):
    examples: List[MsMarcoExample]

    # Load qrels from the provided path and return a dictionary mapping
    # qid -> set({doc_id, doc_id...}) for all doc_ids with relevance over 1.
    @classmethod
    def load_qrels(cls, path: str) -> DefaultDict[str, Set[str]]:
        qrels = defaultdict(set)
        with open(path) as f:
            for line in f:
                qid, _, doc_id, relevance = line.rstrip().split('\t')
                if int(relevance) >= 1:
                    qrels[qid].add(doc_id)
        return qrels

    # Load a run from the provided path.  The run file contains mappings from
    # a query id and a doc title to a rank.  load_run returns a dictionary
    # mapping query ids to lists of doc titles sorted by ascending rank.
    @classmethod
    def load_run(cls, path: str):
        '''Returns OrderedDict[str, List[str]]'''
        run = OrderedDict()
        with open(path) as f:
            for i, line in enumerate(f):
                qid, doc_title, rank = line.split('\t')
                if qid not in run:
                    run[qid] = []
                run[qid].append((doc_title, int(rank)))
        sorted_run = OrderedDict()
        for qid, doc_titles_ranks in run.items():
            doc_titles_ranks.sort(key=lambda x: x[1])
            doc_titles = [doc_titles for doc_titles, _ in doc_titles_ranks]
            sorted_run[qid] = doc_titles
        return sorted_run

    @classmethod
    def load_queries(cls,
                     path: str,
                     qrels: DefaultDict[str, Set[str]],
                     run) -> List[MsMarcoExample]:
        queries = []
        with open(path) as f:
            for line in f:
                qid, query = line.rstrip().split('\t')
                queries.append(MsMarcoExample(qid=qid,
                                              text=query,
                                              candidates=run[qid],
                                              relevant_candidates=qrels[qid]))
        return queries

    @classmethod
    def from_folder(cls,
                    folder: str,
                    split: str = 'dev',
                    is_duo: bool = False,
                    run_path: Path = '.') -> 'MsMarcoDataset':
        run_mono = "mono." if is_duo else ""
        query_path = os.path.join(folder, f"queries.{split}.tsv")
        qrels_path = os.path.join(folder, f"qrels.{split}.tsv")
        if not os.path.isfile(run_path):
            run_path = os.path.join(folder, f"run.{run_mono}{split}.small.tsv")
        return cls(examples=cls.load_queries(query_path,
                                             cls.load_qrels(qrels_path),
                                             cls.load_run(run_path)))

    def query_passage_tuples(self, is_duo: bool = False):
        return [((ex.qid, ex.text, ex.relevant_candidates), perm_pas)
                for ex in self.examples
                for perm_pas in permutations(ex.candidates, r=1+int(is_duo))]

    def to_relevance_examples(self,
                              index_path: str,
                              is_duo: bool = False) -> List[RelevanceExample]:
        loader = MsMarcoPassageLoader(index_path)
        example_map = {}
        for (qid, text, rel_cands), cands in tqdm(self.query_passage_tuples()):
            if qid not in example_map:
                example_map[qid] = [convert_to_unicode(text), [], [], []]
            example_map[qid][1].append([cand for cand in cands][0])
            try:
                passages = [loader.load_passage(cand) for cand in cands]
                example_map[qid][2].append(
                    [convert_to_unicode(passage.all_text)
                     for passage in passages][0])
            except ValueError:
                logging.warning(f'Skipping {passages}')
                continue
            example_map[qid][3].append(cands[0] in rel_cands)
        mean_stats = defaultdict(list)
        for ex in self.examples:
            int_rels = np.array(list(map(int, example_map[ex.qid][3])))
            p = int_rels.sum()/(len(ex.candidates) - 1) if is_duo else int_rels.sum()
            mean_stats['Random P@1'].append(np.mean(int_rels))
            n = len(ex.candidates) - p
            N = len(ex.candidates)
            if len(ex.candidates) <= 1000:
                mean_stats['Random R@1000'].append(1 if 1 in int_rels else 0)
            numer = np.array([sp.comb(n, i) / (N - i) for i in range(0, n + 1) if i != N]) * p
            if n == N:
                numer = np.append(numer, 0)
            denom = np.array([sp.comb(N, i) for i in range(0, n + 1)])
            rr = 1 / np.arange(1, n + 2)
            rmrr = np.sum(numer * rr / denom)
            mean_stats['Random MRR'].append(rmrr)
            rmrr10 = np.sum(numer[:10] * rr[:10] / denom[:10])
            mean_stats['Random MRR@10'].append(rmrr10)
            ex_index = len(ex.candidates)
            for rel_cand in ex.relevant_candidates:
                if rel_cand in ex.candidates:
                    ex_index = min(ex.candidates.index(rel_cand), ex_index)
            mean_stats['Existing MRR'].append(1 / (ex_index + 1)
                                              if ex_index < len(ex.candidates)
                                              else 0)
            mean_stats['Existing MRR@10'].append(1 / (ex_index + 1) if ex_index < 10 else 0)
        for k, v in mean_stats.items():
            logging.info(f'{k}: {np.mean(v)}')
        return [RelevanceExample(Query(text=query_text, id=qid),
                                 list(map(lambda s: Text(s[1], dict(docid=s[0])),
                                          zip(cands, cands_text))),
                                 rel_cands)
                for qid, (query_text, cands, cands_text, rel_cands) in example_map.items()]


In [ ]:
'''dataset = MsMarcoDataset.from_folder(args['dataset'], 
                                     split=args['split'],
                                     is_duo=args['is_duo'],
                                     run_path=args['mono_cache_load_path'])

examples = dataset.to_relevance_examples(args['index_dir'],
                                         is_duo=args['is_duo'])'''

### Load Tokenizer and Ranker model

In [ ]:
options = PassageRankingEvaluationOptions(**args)

construct_map = dict(transformer=construct_transformer,
                      bm25=construct_bm25,
                      seq_class_transformer=construct_seq_class_transformer,
                      random=lambda _: RandomReranker())

reranker = construct_map[args['method']](options)
#writer = MsMarcoWriter(args['output_file'], args['overwrite_output'])

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/58.0 [00:00<?, ?B/s]

In [ ]:
"""evaluator = RerankerEvaluator(reranker, metrics, writer=writer)

width = max(map(len, metrics)) + 1
logging.info("Reranking:")
for metric in evaluator.evaluate(examples):
    logging.info(f'{metric.name:<{width}}{metric.value:.5}')"""

In [ ]:
#simple eval
"""example = examples[0]
score = reranker.rescore(example.query, example.documents)
score[0].score"""

### Run Inference

In [ ]:
import pickle


@torch.no_grad()
def run_tokenizer(reranker, query, text):
      tokens = reranker.tokenizer.encode_plus(query,
                                              text,
                                              max_length=512,
                                              truncation=True,
                                              return_token_type_ids=True,
                                              return_tensors='pt')
      return tokens

@torch.no_grad()
def run_model(reranker,tokens):
    with torch.cuda.amp.autocast(enabled=reranker.use_amp):
        input_ids = tokens['input_ids'].to(reranker.device)
        tt_ids = tokens['token_type_ids'].to(reranker.device)
        output, hidden_states = reranker.model(input_ids, token_type_ids=tt_ids, return_dict=False)
        #hidden = hidden_states[-1]
    return hidden_states
    

for idx, line in enumerate(open(args['dataset']+'/triples.train.small.tsv','r')):
    query, pos_doc, neg_doc = line.split('\t')

    hidden_list = list()

    #positive document
    tokens = run_tokenizer(reranker, query, pos_doc)
    hidden_states = run_model(reranker, tokens)
    hidden_dict = {'query': query,
              'text': pos_doc,
              'label': True,
              'hidden_states': hidden_states.to_device('cpu')}
    hidden_list.append(hidden_dict)

    #negative document
    tokens = run_tokenizer(reranker, query, neg_doc)
    hidden_states = run_model(reranker, tokens)
    hidden = {'query': query,
              'text': neg_doc,
              'label': False,
              'hidden_states': hidden_states.to_device('cpu')}
    hidden_list.append(hidden_dict)

    if line%10000:
        pickle.dump(hidden_list,open('train_hidden_states.pkl','r'))
        !gsutil cp train_hidden_states.pkl gs://luanps/monoBERT/msmarco/

In [ ]:

pickle.dump(hidden_list,open('train_hidden_states_full.pkl','r'))
!gsutil cp train_hidden_states.pkl gs://luanps/monoBERT/msmarco/